In [15]:
import numpy as np

from scipy.spatial import cKDTree
import matplotlib.pyplot as plt


from ultis import *

# Input

In [95]:
image_directory = "datasets/INRIA/images/"
# image_directory = "datasets/UKB/full"

image_paths = get_all_image_paths(image_directory)

In [96]:
# image_paths
len(image_paths) 

1491

# SIFT

In [4]:
all_features = []
for i, path in enumerate(image_paths):
    key_points, descriptors = extract_sift_features(path)
    
    if descriptors is not None:
        # keypoints = sorted(keypoints, key=lambda x: -x.response)
        # max_features = 500
        # keypoints = keypoints[:max_features]
        # descriptors = descriptors[:max_features]
        all_features.append(descriptors)


In [5]:
print(len(all_features))
print(len(image_paths))

1491
1491


In [ ]:

save_all_features(all_features, 1)

In [97]:
all_features2 = load_dataset("checkpoint/dataset_1.npz")

In [ ]:
sum = 0
for i in range(len(all_features)):
    sum += np.sum(all_features[i] - all_features2[i])
sum

# VLAD

In [5]:
from vlad import VLAD

In [6]:
vlad2 = VLAD(k=16, n_vocabs=1, norming="RN", lcs=True)
vlad2.load_model('checkpoint/vlad_k16_ds2/')

c:\Users\thanh\anaconda3\envs\vlad\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## VLAD use

In [31]:
vlad = VLAD(k=64, n_vocabs=1, norming="RN", lcs=True).fit(all_features2)
# vlad = VLAD(k=16, n_vocabs=1, norming="RN", lcs=True).fit(all_features2)

(13695680, 128)
Training vocab #1
Training KMeans...
Finding rotation-matrices...


In [52]:
# vlad_database = vlad._extract_vlads(all_features)
vlad_database = vlad.databases

In [53]:
vlad_database.shape

(10200, 8192)

In [34]:
vlad.save_model(2)

# BOF

In [59]:
from bof import BOF

In [100]:
m_bof = BOF(k=1024)
# m_bof = BOF(k=20480)
m_bof.fit(all_features2)

In [ ]:
m_bof.databases.shape

In [101]:
m_bof.save_model(1)

In [86]:
m_bof.load_model('checkpoint/bof_k1024_ds2/')

# Fisher Vector

In [53]:
from fisher_vector import FisherVector
fivec = FisherVector(n_components=64)
# fivec.fit(all_features2)
fivec.load_model('checkpoint/fish_k64_ds2/')

In [41]:
fivec.save_model(2)

In [54]:
fivec.databases.shape

(10200, 8192)

# ADC

In [75]:
from cq import CQ

In [145]:
cq = CQ(k=16)
cq.fit(vlad_database)
re_vlad_database = cq.residual(vlad_database)

In [146]:
re_vlad_database.shape, vlad_database.shape

((10200, 2048), (10200, 2048))

In [102]:
from adc import ADC

In [103]:
# adc = ADC(m=32, k=1024)
adc = ADC(m=16, k=256)

In [91]:
adc.load_model('checkpoint/adc_k256_m16_ds2_re2/')

In [45]:
adc.fit(fivec.databases)

In [104]:
adc.fit(m_bof.databases)

In [ ]:
adc.fit(vlad_database)
# adc.fit(re_vlad_database)

In [105]:
# adc.load_model('checkpoint/adc_k256_m16_ds1_re3/')
adc.save_model(1, 2)

In [106]:
adc.databases.shape

(1491, 16)

# Test 1

In [107]:
import json
with open('datasets/INRIA/groundtruth.json', 'r') as file:
    data = json.load(file)
    for key in data:
        new_similar = []
        for img in data[key]['similar']:
            for idx, path in enumerate(image_paths):
                if img in path:
                    new_similar.append(idx)
        data[key]['similar'] = new_similar

In [ ]:

outputs = []
for key in data:
    path = 'datasets/INRIA/images/' + data[key]['query']
    key_points, descriptors = extract_sift_features(path)
    # query_vlad = vlad.transform([descriptors])
    
    # re_query_vlad = cq.residual(query_vlad)
    # probabilities = adc.predict_proba(re_query_vlad)
 
    # probabilities = adc.predict_proba(query_vlad)

    query_bof = m_bof.transform([descriptors])
    # re_query_bof = cq.residual(query_bof)

    probabilities = adc.predict_proba(query_bof)
    
    # query_fish = fivec.transform([descriptors])
    # probabilities = adc.predict_proba(query_fish)

    sorted_indices = np.argsort(probabilities)

    output_per_query = []
    for i, index in enumerate(sorted_indices):
        print(f"Original index: {index}, Probability: {probabilities[index]}")
        if i > 0:
            output_per_query.append(index)
            # if len(output_per_query) == len(data[key]['similar']):
            #     break
            if len(output_per_query) == 30:
                break
    
    outputs.append(output_per_query)

Original index: 0, Probability: 0.055902461191864775
Original index: 1, Probability: 0.05800870196590024
Original index: 15, Probability: 0.07030363470484331
Original index: 1392, Probability: 0.07349276001710048
Original index: 1394, Probability: 0.07349276001710048
Original index: 14, Probability: 0.07591073530389782
Original index: 222, Probability: 0.07704071460035214
Original index: 908, Probability: 0.07784903647575539
Original index: 13, Probability: 0.08428602458070684
Original index: 1045, Probability: 0.09522462183983049
Original index: 909, Probability: 0.09585335952752218
Original index: 907, Probability: 0.10304853323126106
Original index: 1044, Probability: 0.10322466568400784
Original index: 1364, Probability: 0.11643686036074057
Original index: 69, Probability: 0.12761095494218277
Original index: 5, Probability: 0.1292383024934175
Original index: 341, Probability: 0.1467750186536112
Original index: 1436, Probability: 0.16373979614360712
Original index: 326, Probability:

In [27]:
from metrics.mAP import compute_map

In [28]:
compute_map(data, outputs)

0.4294221772143204

In [16]:
compute_map(data, outputs)

0.4448341457035376

In [ ]:
compute_map(data, outputs)


In [38]:
compute_map(data, outputs)

0.5263370842598865

In [34]:
compute_map(data, outputs)

0.5446192870841833

In [31]:
compute_map(data, outputs)

0.5446192870841833

In [19]:
compute_map(data, outputs)

0.4353588882737381

In [45]:
# Fisher (k = 16) ADC (m=16, k=256)
compute_map(data, outputs)

0.4323582783182517

In [ ]:
# Fisher (k = 64) ADC (m=64, k=1024)
compute_map(data, outputs)

0.43252457417325424

In [61]:
# VLAD (k=16) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.5376465998449635

In [ ]:
# VLAD (k=64) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

In [70]:
# BOF (k=1024) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.41684722833359034

In [ ]:
# BOF (k=1024) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

0.45380486841201123

In [83]:
# BOF (k=20480) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

0.3191882686872316

In [88]:
# BOF (k=20480) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.35210481160541585

# Test 2

In [27]:
from metrics.score4 import calculate_gr_score

In [93]:
similarity_matrix = []
for path in image_paths:
    key_points, descriptors = extract_sift_features(path)
    
    # query_fish = fivec.transform([descriptors])
    # probabilities = adc.predict_proba(query_fish)
    query_bof = m_bof.transform([descriptors])
    probabilities = adc.predict_proba(query_bof)

    # query_vlad = vlad2.transform([descriptors])
    # # re_query_vlad = cq.residual(query_vlad)
    # probabilities = adc.predict_proba(query_vlad)
    
    similarity_matrix.append(probabilities)

In [81]:
len(similarity_matrix[0]), len(similarity_matrix)

(10200, 10200)

In [94]:
calculate_gr_score(np.array(similarity_matrix))

2.089313725490196

In [57]:
calculate_gr_score(np.array(similarity_matrix))

1.6491176470588236

In [47]:
# BOF k=1024 ADC m=16 k=256
calculate_gr_score(np.array(similarity_matrix))

2.082549019607843

In [39]:
# BOF k=1024 ADC m=32 k=1024
calculate_gr_score(np.array(similarity_matrix))

2.2905882352941176

In [26]:
# Fisher k=64 ADC m=32 k=1024
calculate_gr_score(np.array(similarity_matrix))

2.0431372549019606

In [ ]:
# Fisher k=16 ADC m=16 k=256
calculate_gr_score(np.array(similarity_matrix))

2.241862745098039

# KDTree

In [ ]:
kdtree=cKDTree(vlad.databases)

In [ ]:
def display_image_with_vector(image_path, vlad_vector, title):
    """
    Displays an image with its descriptor vector.

    Args:
        image_path (str): Path to the image.
        vlad_vector (numpy.ndarray): VLAD vector.
        title (str): Title for the plot.
    """
    image = cv2.imread(image_path)
    plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()


# Function to find and display similar images
def search_similar_images(query_vector, kdtree, vlad_vectors_reduced,
                          num_results=1):
    """
    Finds and displays similar images based on a query vector.

    Args:
        query_vector (numpy.ndarray): Query vector.
        kdtree: KD-Tree built on the training set.
        vlad_vectors_reduced: Reduced VLAD vectors for the training set.
        num_results (int): Number of similar images to display.
    """
    _, indices = kdtree.query(query_vector, num_results)
    print(indices)

    if num_results == 1:
        idx = int(indices[0])  # Access the first index
        image_path = image_paths[idx]
        display_image_with_vector(image_path,
                                  vlad_vectors_reduced[idx], "Similar Image")
    else:
        for i in range(num_results):
            idx = int(indices[i])
            image_path = image_paths[idx]
            display_image_with_vector(image_path,
                                      vlad_vectors_reduced[idx], f"Similar Image {i + 1}")


# Function to find the path of a similar image
def search_similar_images_path(query_vector, kdtree, vlad_vectors_reduced,
                               num_results=1):
    """
    Finds the path of a similar image based on a query vector.

    Args:
        query_vector (numpy.ndarray): Query vector.
        kdtree: KD-Tree built on the training set.
        vlad_vectors_reduced: Reduced VLAD vectors for the training set.
        num_results (int): Number of similar images to find.

    Returns:
        str: Path of the similar image.
    """
    _, indices = kdtree.query(query_vector, num_results)

    if num_results == 1:
        idx = int(indices[0])  # Access the first index
        image_path = image_paths[idx]
        return image_path
    else:
        for i in range(num_results):
            idx = int(indices[i])
            image_path = image_paths[idx]
            return image_path

In [ ]:
vlad.database.shape

In [ ]:
search_similar_images(vlad.database[0], kdtree, vlad.database, 3)